# Validations

## Bmh knockout

Source: https://github.com/ntumitolab/MitoRetroDynamics/blob/main/validation.ipynb

Initial protein concentrations: Get expression levels: uses data from [GSE102475](https://github.com/NTUMitoLab/RetroSignalModel.jl?organization=NTUMitoLab&organization=NTUMitoLab#differential-gene-expression).

In [ ]:
using StatsPlots
using RetroSignalModel
using ModelingToolkit
using DifferentialEquations

import RetroSignalModel as rs

In [ ]:
proteinwt = rs.STRESSED
proteinΔbmh = Dict(proteinwt..., rs.ΣBmh=>1e-4)

Let's try one set of params and solve the steady-state solution:

In [ ]:
@named sys = RtgMTK(rs.ONE_SIGNAL)
u0 = rs.resting_u0(sys)

kinconstants = load_parameters("solution_rtgM4.csv")[1]
params = merge(proteinwt, kinconstants, Dict(rs.mul_S => 0.1))

prob = SteadyStateProblem(sys, u0, params)
sol = solve(prob, DynamicSS(Rodas5()))

Considering a combination of Wild-type vs. Bmh knockout, and Low damage signal = `0.1` vs. High = `0.9`.

In [ ]:
sol_wt_lo = solve(SteadyStateProblem(sys, u0, merge(params, Dict(rs.mul_S => 0.1))), DynamicSS(Rodas5()))
sol_wt_hi = solve(SteadyStateProblem(sys, u0, merge(params, Dict(rs.mul_S => 0.9))), DynamicSS(Rodas5()))
sol_Δb_lo = solve(SteadyStateProblem(sys, u0, merge(params, proteinΔbmh, Dict(rs.mul_S => 0.1))), DynamicSS(Rodas5()))
sol_Δb_hi = solve(SteadyStateProblem(sys, u0, merge(params, proteinΔbmh, Dict(rs.mul_S => 0.9))), DynamicSS(Rodas5()))

In [ ]:
r3nu = [rs.rtg3_nucleus(sol) for sol in (sol_wt_lo, sol_wt_hi, sol_Δb_lo, sol_Δb_hi)]
r3cy = [rs.rtg3_cytosol(sol) for sol in (sol_wt_lo, sol_wt_hi, sol_Δb_lo, sol_Δb_hi)]

ticklabel = ["WT (Low S)", "WT (Hi S)", "ΔBmh (Low S)", "ΔBmh (Hi S)"]

groupedbar([r3nu r3cy], 
    bar_position = :stack, 
    bar_width=0.5,
    xticks=(1:length(ticklabel), ticklabel),
    label=["Rtg3 (nucleus)" "Rtg3 (cytosol)"],
    title="Rtg3 translocation",
    ylabel = "Rtg3 concnetration",
    legend=:right
    )

## Rapamycin and glutamate

`kn2M` was decreased ten times to simulated enhanced binding of Mks and Rtg2 upon Rapamycin / glutamate addition.

In [ ]:
param_rapamycin = copy(params)
param_rapamycin[rs.kn2M] /= 10

@named sys = RtgMTK(rs.ONE_SIGNAL)
u0 = rs.resting_u0(sys)

sol_ra_lo = solve(SteadyStateProblem(sys, u0, merge(param_rapamycin, Dict(rs.mul_S => 0.1))), DynamicSS(Rodas5()))
sol_ra_hi = solve(SteadyStateProblem(sys, u0, merge(param_rapamycin, Dict(rs.mul_S => 0.9))), DynamicSS(Rodas5()))

In [ ]:
r3nu = [rs.rtg3_nucleus(sol) for sol in (sol_wt_lo, sol_wt_hi, sol_ra_lo, sol_ra_hi)]
r3cy = [rs.rtg3_cytosol(sol) for sol in (sol_wt_lo, sol_wt_hi, sol_ra_lo, sol_ra_hi)]

ticklabel = ["Control (Low S)", "Control (Hi S)", "Rapa (Low S)", "Rapa (Hi S)"]

groupedbar([r3nu r3cy], 
    bar_position = :stack, 
    bar_width=0.5,
    xticks=(1:length(ticklabel), ticklabel),
    label=["Rtg3 (nucleus)" "Rtg3 (cytosol)"],
    title="Rtg3 translocation",
    ylabel = "Rtg3 concnetration",
    legend=:right
    )